In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# =====================
# 1. LOAD DATA
# =====================
df = pd.read_csv("flash_flood_with_labels.csv")

# แปลง timestamp เป็น datetime (รองรับหลายรูปแบบ)
try:
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%m/%Y %H:%M')
except:
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M')
    except:
        df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

print(f"Loaded {len(df)} rows")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

Loaded 1548 rows
Date range: 2024-06-10 13:00:00 to 2024-10-01 22:15:00


In [12]:
for window in [3, 6, 9]:
    df[f'rain_sum_{window}'] = df.groupby('sensor_id')['rainfall_mm_per_hr'].rolling(window=window, min_periods=1).sum().reset_index(level=0, drop=True)
    df[f'rain_std_{window}'] = df.groupby('sensor_id')['rainfall_mm_per_hr'].rolling(window=window, min_periods=1).std().reset_index(level=0, drop=True)
    df[f'flow_max_{window}'] = df.groupby('sensor_id')['water_flow_rate_m3_per_sec'].rolling(window=window, min_periods=1).max().reset_index(level=0, drop=True)
    df[f'vibration_mean_{window}'] = df.groupby('sensor_id')['vibration_magnitude'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

# Rate of change (อัตราการเปลี่ยนแปลง)
df['rain_change'] = df.groupby('sensor_id')['rainfall_mm_per_hr'].diff()
df['flow_change'] = df.groupby('sensor_id')['water_flow_rate_m3_per_sec'].diff()
df['vibration_change'] = df.groupby('sensor_id')['vibration_magnitude'].diff()

# Lag features (ค่าจากช่วงเวลาก่อนหน้า)
for col in ['rainfall_mm_per_hr', 'water_flow_rate_m3_per_sec', 'vibration_magnitude']:
    df[f'{col}_lag1'] = df.groupby('sensor_id')[col].shift(1)
    df[f'{col}_lag2'] = df.groupby('sensor_id')[col].shift(2)

# Interaction features
df['rain_flow_interaction'] = df['rainfall_mm_per_hr'] * df['water_flow_rate_m3_per_sec']
df['rain_vibration_interaction'] = df['rainfall_mm_per_hr'] * df['vibration_magnitude']

# Sensor-specific features
df['sensor_upstream'] = (df['sensor_name'] == 'Upper').astype(int)
df['sensor_midstream'] = (df['sensor_name'] == 'Middle').astype(int)

# Handle missing values from lag/diff operations
df = df.fillna(method='bfill').fillna(0)

print(f"Created {len(df.columns)} features")

Created 39 features


C:\Users\boatd\AppData\Local\Temp\ipykernel_7060\3233268443.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill').fillna(0)


In [14]:
# =====================
# 3. PREPARE DATASETS
# =====================
# Define feature columns (exclude timestamp, IDs, and target)
feature_cols = [col for col in df.columns if col not in 
                ['timestamp', 'sensor_id', 'sensor_name', 'event_intensity', 
                 'is_flash_flood', 'time_to_next_sensor_min']]

X = df[feature_cols]
y_classification = df['is_flash_flood']

# For regression, only use rows with valid time_to_next_sensor
df_reg = df[df['time_to_next_sensor_min'].notna()].copy()
X_reg = df_reg[feature_cols]
y_reg = df_reg['time_to_next_sensor_min']

# Temporal split (70% train, 15% val, 15% test)
split_date_1 = df['timestamp'].quantile(0.70)
split_date_2 = df['timestamp'].quantile(0.85)

train_mask = df['timestamp'] <= split_date_1
val_mask = (df['timestamp'] > split_date_1) & (df['timestamp'] <= split_date_2)
test_mask = df['timestamp'] > split_date_2

X_train = X[train_mask]
X_val = X[val_mask]
X_test = X[test_mask]

y_train = y_classification[train_mask]
y_val = y_classification[val_mask]
y_test = y_classification[test_mask]

print(f"\nClassification Split:")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"Train flood ratio: {y_train.mean():.2%}")


# Quick checks: class distribution in each split
print("\nClass distributions:")
print("Train:", y_train.value_counts(dropna=False).to_dict())
print("Val:  ", y_val.value_counts(dropna=False).to_dict())
print("Test: ", y_test.value_counts(dropna=False).to_dict())

# If any split contains only one class, fallback to stratified random split
if len(y_train.unique()) < 2 or len(y_val.unique()) < 2 or len(y_test.unique()) < 2:
    print("\nWarning: One or more temporal splits contain only a single class. Falling back to stratified random split.")
    from sklearn.model_selection import train_test_split
    X_temp, X_test, y_temp, y_test = train_test_split(X, y_classification, test_size=0.15, stratify=y_classification, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1764705882, stratify=y_temp, random_state=42)  # 0.17647*0.85 ≈ 0.15 overall
    print("New splits (stratified):")
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print("Train flood ratio:", y_train.mean())


Classification Split:
Train: (1083, 33), Val: (234, 33), Test: (231, 33)
Train flood ratio: 70.08%

Class distributions:
Train: {1: 759, 0: 324}
Val:   {1: 234}
Test:  {1: 231}

New splits (stratified):
Train: (1082, 33), Val: (233, 33), Test: (233, 33)
Train flood ratio: 0.7911275415896488


In [15]:
# =====================
# 4. TRAIN CLASSIFICATION MODEL
# =====================
print("\n" + "="*50)
print("TRAINING CLASSIFICATION MODEL")
print("="*50)

# Try multiple models

best_model_name = None
best_score = -1
models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=300,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
}

best_model = None
best_model_name = None 
best_score = -1

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    
    # Validation
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]

    if hasattr(model, "predict_proba"):
        y_val_proba = model.predict_proba(X_val)[:, 1]
    
    # Compute ROC AUC only if validation has both classes and model provides probabilities
    if y_val_proba is not None and len(np.unique(y_val)) == 2:
        roc_auc = roc_auc_score(y_val, y_val_proba)
        print(f"{name} - ROC AUC: {roc_auc:.4f}")
    else:
        roc_auc = np.nan
        print(f"{name} - ROC AUC: nan (validation set does not contain both classes or model has no predict_proba)")
    
    # roc_auc = roc_auc_score(y_val, y_val_proba)
    # print(f"{name} - ROC AUC: {roc_auc:.4f}")
    
    if roc_auc > best_score or best_model is None:
        best_score = roc_auc
        best_model = model
        best_model_name = name

print(f"\nBest model: {best_model_name} (ROC AUC: {best_score:.4f})")

# Final evaluation on test set
y_test_pred = best_model.predict(X_test)
y_test_proba = best_model.predict_proba(X_test)[:, 1]

if hasattr(best_model, "predict_proba"):
    y_test_proba = best_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print("FINAL TEST SET RESULTS - CLASSIFICATION")
print("="*50)
print(classification_report(y_test, y_test_pred))
print(f"ROC AUC: {roc_auc_score(y_test, y_test_proba):.4f}")

if y_test_proba is not None and len(np.unique(y_test)) == 2:
    print(f"ROC AUC: {roc_auc_score(y_test, y_test_proba):.4f}")
else:
    print("ROC AUC: nan (test set does not contain both classes or model has no predict_proba)")

# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix:")
print(cm)

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))
joblib.dump(best_model, 'flash_flood_classifier.pkl')
print("\nSaved model: flash_flood_classifier.pkl")


TRAINING CLASSIFICATION MODEL

Training RandomForest...
RandomForest - ROC AUC: 1.0000

Training GradientBoosting...
GradientBoosting - ROC AUC: 1.0000

Best model: RandomForest (ROC AUC: 1.0000)

FINAL TEST SET RESULTS - CLASSIFICATION
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        49
           1       1.00      1.00      1.00       184

    accuracy                           1.00       233
   macro avg       1.00      1.00      1.00       233
weighted avg       1.00      1.00      1.00       233

ROC AUC: 1.0000
ROC AUC: 1.0000

Confusion Matrix:
[[ 49   0]
 [  0 184]]

Top 10 Most Important Features:
                feature  importance
7                 month    0.390366
6           day_of_week    0.103553
19     vibration_mean_9    0.081419
5                  hour    0.062518
15     vibration_mean_6    0.061518
17           rain_std_9    0.045187
11     vibration_mean_3    0.037844
18           flow_max_9    0.036101
13  

In [16]:
# =====================
# 5. TRAIN REGRESSION MODEL (Time to Next Sensor)
# =====================
print("\n" + "="*50)
print("TRAINING REGRESSION MODEL")
print("="*50)

# Temporal split for regression data
train_mask_reg = df_reg['timestamp'] <= split_date_1
val_mask_reg = (df_reg['timestamp'] > split_date_1) & (df_reg['timestamp'] <= split_date_2)
test_mask_reg = df_reg['timestamp'] > split_date_2

X_train_reg = X_reg[train_mask_reg]
X_val_reg = X_reg[val_mask_reg]
X_test_reg = X_reg[test_mask_reg]

y_train_reg = y_reg[train_mask_reg]
y_val_reg = y_reg[val_mask_reg]
y_test_reg = y_reg[test_mask_reg]

print(f"Regression Split:")
print(f"Train: {X_train_reg.shape}, Val: {X_val_reg.shape}, Test: {X_test_reg.shape}")

# Train regression model
reg_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=10,
    min_samples_split=15,
    min_samples_leaf=10,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

reg_model.fit(X_train_reg, y_train_reg)

# Validation
y_val_reg_pred = reg_model.predict(X_val_reg)
val_mae = mean_absolute_error(y_val_reg, y_val_reg_pred)
val_r2 = r2_score(y_val_reg, y_val_reg_pred)
print(f"\nValidation - MAE: {val_mae:.2f} min, R²: {val_r2:.4f}")

# Test set evaluation
y_test_reg_pred = reg_model.predict(X_test_reg)
test_mae = mean_absolute_error(y_test_reg, y_test_reg_pred)
test_r2 = r2_score(y_test_reg, y_test_reg_pred)

print("\n" + "="*50)
print("FINAL TEST SET RESULTS - REGRESSION")
print("="*50)
print(f"MAE: {test_mae:.2f} minutes")
print(f"R²: {test_r2:.4f}")
print(f"RMSE: {np.sqrt(np.mean((y_test_reg - y_test_reg_pred)**2)):.2f} minutes")
joblib.dump(reg_model, 'flash_flood_time_regressor.pkl')
print("\nSaved model: flash_flood_time_regressor.pkl")


TRAINING REGRESSION MODEL
Regression Split:
Train: (1083, 33), Val: (234, 33), Test: (231, 33)

Validation - MAE: 2.27 min, R²: -9.0192

FINAL TEST SET RESULTS - REGRESSION
MAE: 1.45 minutes
R²: -21.1575
RMSE: 1.55 minutes

Saved model: flash_flood_time_regressor.pkl


In [56]:
# =====================
# 6. INFERENCE EXAMPLE
# =====================
print("\n" + "="*50)
print("INFERENCE EXAMPLE")
print("="*50)

# Take a sample from test set
# sample = X_test.iloc[0:1]
# flood_prob = best_model.predict_proba(sample)[0, 1]
# flood_pred = "YES" if flood_prob > 0.5 else "NO"

sample_idx = 0
sample = X_test.iloc[sample_idx:sample_idx+1]
sensor_name = df.loc[sample.index[0], 'sensor_name']

flood_prob = best_model.predict_proba(sample)[0, 1]
flood_pred = "YES" if flood_prob > 0.5 else "NO"

print(f"Location: {sensor_name} sensor")
print(f"Flash Flood Prediction: {flood_pred} (Probability: {flood_prob:.2%})")

# If flood predicted and sample has next sensor info, predict time
if flood_pred == "YES" and test_mask_reg.iloc[sample_idx]:
    sample_reg = X_reg[test_mask_reg].iloc[sample_idx:sample_idx+1]
    next_sensor = "Middle" if sensor_name == "Upper" else "Lower"
    time_pred = reg_model.predict(sample_reg)[0]
    print(f"Estimated Time to {next_sensor} Sensor: {time_pred:.1f} minutes")

print("\n" + "="*50)
print("DONE!")
print("="*50)


INFERENCE EXAMPLE
Location: Upper sensor
Flash Flood Prediction: NO (Probability: 11.05%)

DONE!
